In [65]:
import json
import jsonlines
import os
import subprocess
from tqdm import tqdm


In [85]:
system_prompt = """
You are an intelligent assistant designed to help insert potential items into a user's interaction history sequence.
Given a user's interaction history and a list of potential items, your task is to assess whether it makes sense to insert any of the potential items into the user's history sequence.

Your process is as follows:
1. Analyze the items in the user's interaction history to identify their most recent area of interest.
2. Evaluate whether inserting one or more potential items would reasonably reflect the user’s most recent interests or important interest, while maintaining coherence in the sequence.
3. If it is reasonable to insert potential items, generate an updated sequence with the inserted item(s) placed in the appropriate position. 
If it is not reasonable, provide the reason and return a `null` sequence. 
Only generate an updated sequence when you are confident in the decision.

When generating the updated sequence, return it in JSON format, including both `asin` and `timestamp`. 
The timestamp for the inserted items should be determined based on their relationship with the adjacent items, leaning towards the more similar item.
"""

human_prompt_template = """
User Interaction History:
{history_list}

Potential Items:
{potential_items}

Please determine whether any potential items should be inserted into the user's history. If so, generate the updated sequence with appropriate timestamps.

# <FORMAT>
Your output should be in JSON format:
"reason": your reason,
"potential_sequence": [
    {{
        "asin": "<asin>",
        "timestamp": <timestamp>
    }}
] or null
# </FORMAT>

# <Your reason and potential_sequence>
"""

output_tempate = """
"reason": {reason},
"potential_sequence": {potential_sequence}
"""

In [2]:
input_file = "/mnt/hwfile/internvideo/share_data/wuyue/data/SeqRec/toy_file_1_updated.json"
output_file = "/mnt/hwfile/internvideo/share_data/wuyue/data/SeqRec/output-1200.json"

with open(input_file, 'r') as file1, open(output_file, 'r') as file2:
    input_data_dict = json.load(file1)
    output_data_dict = json.load(file2)

    

In [3]:
len(input_data_dict), len(output_data_dict)

(9706, 1186)

In [6]:
input_data_dict.keys()

dict_keys(['AEWFOZKLII3T8', 'A2DIQBZ1VZCK09', 'A2ZM09SXQD77H0', 'A3BL564DYD9MJY', 'A2RAHVQXI12VGA', 'A2EHQDD4X4SLRJ', 'AF41ELZ2DWMV6', 'A1AYVGYE7V4T93', 'A2OUN2538DD7S2', 'A2H4KS3K0ZSF2V', 'A2XC8MF14WTYZ2', 'A284P8XSG9HREA', 'A395L0BRXXI9J3', 'AXR3CV6A4FMBG', 'AVXO3FIFX98DU', 'A51TSKKQAA3ET', 'A1VBD25ZDDVKKR', 'A3QVAKVRAH657N', 'ALCP5UW9GADFI', 'A27C84PT7QTJDC', 'A3BHZSE1OQCANO', 'A1UNI64M36ZLFC', 'AJR8OLVMD39I7', 'A1LPV8DN13DI8K', 'A2M4JAYE4B7UQ4', 'A2PRFCYJQRLTTN', 'A1GF0Q8VRMJDNF', 'A36RK0WE9U57BZ', 'A2JZ8T64YZDQSI', 'A1KM020ZU2RMP7', 'AXTPMJJCW4I7I', 'A32U5F3BYVM4NA', 'ADBUCOBPEOL15', 'ASWHAWQ4Q1NF7', 'ARZ2SH4Q1OFO1', 'A27ZFNMC15SHJD', 'A4V72SI1REZ3W', 'A2UTQXA2X3Q1TG', 'A2TJSDKB71NJAT', 'A3KZR2JX15PX62', 'AXHETL0M8XZPT', 'A3GALEUCJLVQMP', 'A1R8TTMXVXEI6H', 'A3BTJ55E2N70KP', 'AIWGYBEVV09B1', 'A1X93ES4DITTWK', 'ATAGGWE8OHDJS', 'A2X6ZNP4SL5X75', 'AQJH1WMWQAIWQ', 'A978N3TH1DH6Y', 'A3QOAEXTCJAU13', 'A2KWIRXYFC1OQ1', 'A2H2T7YX7FE04D', 'A36RJG7HHV45TQ', 'A3L0HOM7SL74MJ', 'A2CYXQOAR1EJRQ'

In [8]:
output_data_dict.keys()

dict_keys(['0000000000002', '0000000000003', '0000000000004', '0000000000005', '0000000000006', '0000000000007', '0000000000008', '0000000000009', '0000000000010', '0000000000011', '0000000000012', '0000000000013', '0000000000014', '0000000000015', '0000000000016', '0000000000017', '0000000000018', '0000000000019', '0000000000020', '0000000000021', '0000000000022', '0000000000023', '0000000000024', '0000000000025', '0000000000026', '0000000000027', '0000000000028', '0000000000029', '0000000000030', '0000000000031', '0000000000032', '0000000000033', '0000000000034', '0000000000035', '0000000000036', '0000000000037', '0000000000038', '0000000000039', '0000000000040', '0000000000041', '0000000000042', '0000000000043', '0000000000044', '0000000000045', '0000000000046', '0000000000047', '0000000000048', '0000000000049', '0000000000050', '0000000000051', '0000000000052', '0000000000053', '0000000000054', '0000000000055', '0000000000056', '0000000000057', '0000000000058', '0000000000059', '00

In [71]:
input_data_dict['AEWFOZKLII3T8'].keys()
# len(input_data_dict['AEWFOZKLII3T8']['history_list'])
# len(input_data_dict['AEWFOZKLII3T8']['potential_items'])
# input_data_dict['A2DIQBZ1VZCK09']['history_list']

dict_keys(['history_list', 'potential_items'])

In [75]:
output_data_dict['0000000000002'].keys()
# output_data_dict['0000000000003']['history_list']
# (output_data_dict['0000000000002']['potential_sequence'])
# output_data_dict['0000000000002']

dict_keys(['history_list', 'potential_sequence', 'reason'])

In [18]:
def are_dicts_equal(dict1, dict2):
    # 首先检查键的集合是否相等
    if set(dict1.keys())!= set(dict2.keys()):
        return False
    # 再检查每个键对应的值是否相等
    for key in dict1.keys():
        if dict1[key]!= dict2[key]:
            return False
    return True

In [66]:
input_output_list = []

i = j = 0

last_found = 0

for j in tqdm(range(len(output_data_dict))):
    output_key = list(output_data_dict.keys())[j]
    
    for i in range(len(input_data_dict)):
        flag = 1
        input_key = list(input_data_dict.keys())[i]
        for item in output_data_dict[output_key]['history_list']:
            if not item in input_data_dict[input_key]['history_list']:
                flag = 0
        if flag == 1:
            input_output_list.append([input_key, output_key])
            break


100%|██████████| 1186/1186 [01:37<00:00, 12.21it/s]


In [67]:
len(input_output_list)

1186

In [81]:
total_label_ls = []

for input_key, output_key in input_output_list:
    tmp_dict = {}
    tmp_dict['id'] = input_key
    tmp_dict['history_list'] = input_data_dict[input_key]['history_list']
    tmp_dict['potential_items'] = input_data_dict[input_key]['potential_items']
    tmp_dict['potential_sequence'] = output_data_dict[output_key]['potential_sequence']
    tmp_dict['reason'] = output_data_dict[output_key]['reason']
    total_label_ls.append(tmp_dict)

In [87]:
total_label_ls[0].keys()

dict_keys(['id', 'history_list', 'potential_items', 'potential_sequence', 'reason'])

In [90]:
train_json_ls = []

for data in total_label_ls:
    tmp_dict = {}
    tmp_dict['id'] = data['id']
    tmp_dict['instruction'] = system_prompt
    tmp_dict['input'] = human_prompt_template.format(
        history_list=json.dumps(data['history_list'], indent=2),
        potential_items=json.dumps(data['potential_items'], indent=2)  # Pass the whole list of potential items
    )
    tmp_dict['output'] = output_tempate.format(reason=data['reason'], potential_sequence=json.dumps(data['potential_sequence'], indent=2))
    train_json_ls.append(tmp_dict)

In [92]:
with open('/mnt/hwfile/internvideo/share_data/wuyue/data/SeqRec/train.json', 'w') as f:
    json.dump(train_json_ls, f, ensure_ascii=False, indent=2)